In [1]:
%cd ..

/data/home/eak/learning/llm_finetuning/specializing-llm-telecom


In [2]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

MODEL_PATH = "unsloth/llama-3-8b-Instruct-bnb-4bit"  #"unsloth/gemma-1.1-2b-it" # "unsloth/llama-3-8b-Instruct-bnb-4bit"
# MODEL_PATH = "unsloth/tinyllama-bnb-4bit"

In [3]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer
from peft import PeftModelForCausalLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


result: tuple[PeftModelForCausalLM, AutoTokenizer] = FastLanguageModel.from_pretrained(
	model_name = MODEL_PATH,
	max_seq_length = max_seq_length,
	dtype = dtype,
	load_in_4bit = load_in_4bit,
	# token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
model: PeftModelForCausalLM = result[0]
tokenizer: AutoTokenizer = result[1]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.438 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.1+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.23+cu118. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
import json

QUERY = """
Please generate <num_questions>{NUM_QUESTIONS}</num_questions> questions based on the provided context about 3GPP (3rd Generation Partnership Project, standards for 3G (third generation), 4G (fourth generation), and 5G (fifth generation) mobile networks).

Carefully analyze the given context and create questions that can be unambiguously answered using only the information provided. Avoid questions that would require external knowledge to answer.

For each question you create:
- Enclose the question text within "question" quotes.
- Generate 5 plausible answer options based on the context, labeling them as "option 1", "option 2", etc.
- Indicate the correct answer inside "answer" quotes, referencing the option number.
- Provide a short explanation supporting the correct answer within "context" quotes.

Output each complete question with its answer options and correct answer inside <qa> tags, using this format:

<qa>
question: Question text goes here
option 1: First answer option
option 2: Second answer option
option 3: Third answer option
option 4: Fourth answer option
option 5: Fifth answer option
answer: option X: correct answer text
context: Brief explanation supporting the correct answer
</qa>

Additional guidelines:
- Generate unique questions without repetition.
- Avoid overusing phrases like "What is the purpose of..." when formulating questions.
- Create diverse question types (e.g., multiple-choice, true/false, fill-in-the-blank) to test different aspects of understanding.
- Context has been randomly extracted from the document so, number at the beginning of the text may be related to section number, and not directly related to the context
- Focus on key concepts, technologies, and processes related to 3GPP mentioned in the context.
- Include questions about specific standards, releases, or technical specifications if mentioned.
- When appropriate, ask about relationships between different elements or compare and contrast various aspects of 3GPP.
- Avoid adding extra text at the end or the begining of generation

Here is the context to generate questions from:

<context>
{CONTEXT}
</context>
"""

In [20]:
NUM_QUESTIONS = 5

CONTEXT = """
The " Integration of satellite components in the 5G architecture” work item adds or enhances a number of features in 5GCore architecture in order to support Non-Terrestrial Networks (NTN), for several use cases:

Coverage extension: Many commercial activities, such as agriculture, mining, forestry take place outside inhabited areas. Coverage extension with satellite networks is useful to enable e.g. voice communication, video monitoring, and remote control in uncovered or under-covered areas. 

Internet of Things: many Internet-of-Things applications relate to monitoring of assets (e.g. ships, trains, trucks), infrastructure (e.g. bridges, pipelines, railway track), or the environment (e.g. agriculture sensors). In many IoT applications, only small amounts of data are exchanged and communication is optimized for low power usage. Satellite communication should also be able to address these requirements.

Disaster communication: Public safety authorities have a responsibility to provide assistance in case of natural disasters. This requires communication, also in cases where because of that disaster the cellular infrastructure is damaged. Satellite communication can be used as fall back for these cases. Ideally the user equipment (UE) and way of working when cellular networks are available should also be usable with satellite access.

Global roaming: Applications like tracking and tracing of containers need to be available globally across satellite and terrestrial networks. When a container is in a harbour or transported on a truck, using a terrestrial cellular network is probably most efficient. However, when the container is on a ship in the middle of an ocean, only satellite communication is possible.

Broadcasting: Satellite communication is particularly suitable to broadcast the same information over a very wide area. This can also be used in context of 5G mobile edge applications (e.g. mobile gaming), where application content needs to be available in many different edge locations.

To address such use cases, 3GPP has set Key Performance Indicator (KPI) targets for satellite in TS 22.261 [1].

At 5G Core Network architecture level, in SA2, a dedicated study on architecture aspects for using satellite access in 5G (FS_5GSAT_ARCH) was conducted to select the solutions able to cope with satellite specific key issues. The outcome of the study (TR 23.737 [2]) identifies the impacts of satellite integration in the 5GS and solutions to adjust the 5G system accordingly. 

The 5GSAT_ARCH work item, following the study, updated architecture specifications (TS 23.501 [4], TS 23.502 [5], TS 23.503 [6]) to implement the solutions identified. 

In CT1, TR 24.821 [3] studied "Non-Terrestrial Impact of PLMN selection procedure" and, following 5GSAT_ARCH_CT, led to update TS 23.122 [7] and TS 24.501 [8]. 

Furthermore, RAN has defined "3GPP defined radio access networks supporting non-Terrestrial Networks" [10], described in the next clause.

Architectural/general aspects

A PLMN core network can be connected to a satellite NG-RAN. A satellite NG-RAN can be shared between more than one core networks. 

Satellite NG-RAN can be used as a new RAN 3GPP access but also as backhaul between the core and terrestrial access network, providing a transport for the N1/N2/N3 reference points.

Multi-connectivity and URLLC over satellite are not considered in Rel-17. Basic assumptions are that UEs are equipped with GNSS, and transparent mode: satellites (LEO/MEO/GEO) are relaying the Uu interface only at physical layer level.

Impacts on 5GC of Satellite NG-RAN used as new RAN 3GPP access

In Rel-17, only direct access with transparent satellite is considered, as shown in following figure:

 

Figure 1: Direct access with transparent satellite

Impacts of satellites onto 5GS are linked to the size of the cells (larger than the terrestrial ones), the fact that the satellite cells can be fix on earth, when beam is steerable, but also moving on earth, when beam is not steerable. This characteristics impacts 5GS mobility management, i.e. the management of the handover of radio bearer between nodes and the management of the reachability of a UE for downlink services (paging), that need to be adapted to take into account both the satellite beam size and fix or moving cells configuration. 

A basic assumption in Re-17 is that tracking areas (TAs) and cell identities (cell IDs) refer to specific geographical areas, so that 5G services can use these identifiers as representation of a UE location. 

To avoid Tracking Area Codes (TAC) fluctuations, in the moving cells case, it has been decided that the Radio Access Network will broadcast in the cell the list of Tracking Area Codes, corresponding to tracking areas that have been define on the earth surface through network planning, for the zone currently enlighten by the radio cell. 

New Radio Access Technology types are introduced in the 5GC to distinguish between different satellite configurations (LEO, MEO, GEO, other).

The distance earth – satellite also introduces higher delay values than for terrestrial cells and new 5QI is also introduced in TR 23.501 [4] to cope with this delay, depending on the satellite RAT type.  

Impacts on 5GC of Satellite NG-RAN used as backhaul

Rel-17 only considers backhauling with constant delay. Here, the satellite operator is able to mask any delay changes in service/feeder links by exploiting the knowledge of the satellite position to calculate how much variable delay should be added to keep the overall delay constant. Connecting gNBs to 5GC via, e.g., a single GEO satellite or a single NGSO satellite without ISLs are examples of such backhauling as shown in following figure.
"""

In [7]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

In [21]:
text = QUERY.format(NUM_QUESTIONS=NUM_QUESTIONS, CONTEXT=CONTEXT)

len(text)

7852

In [22]:
chat = [
	{"role": "user", "content": text},
]
prompt = tokenizer.apply_chat_template(
	chat, tokenize=False, add_generation_prompt=True
)

In [23]:
inputs = tokenizer(
	prompt, return_tensors = "pt", padding=True, truncation=True
).to("cuda")

In [24]:
predicted = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2096,)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here are 5 questions based on the provided context about 3GPP and satellite networks:

<qa>
question: What is the primary purpose of the "Integration of satellite components in the 5G architecture" work item?
option 1: To improve 5G network security
option 2: To support Non-Terrestrial Networks (NTN) and various use cases
option 3: To enhance 5G network performance
option 4: To reduce latency in 5G networks
option 5: To increase network coverage
answer: option 2
context: The work item aims to add or enhance features in 5G Core architecture to support Non-Terrestrial Networks (NTN) and various use cases, such as coverage extension, Internet of Things, disaster communication, global roaming, and broadcasting.
</qa>

<qa>
question: What is the primary use case for satellite communication in disaster scenarios?
option 1: Providing high-speed internet access
option 2: Enabling voice communication and video monitoring
option 3: Supporting remote control of critical infrastructure
option 4: P

In [12]:
CONTEXT = """
Introduction

Clause 6.1 explains the relationship in Rel-14 between TS 22.179 (MCPTT), TS 22.280 (MCCoRe), TS 22.281 (MCVideo) and TS 22.282 (MCData).

This clause deals more specifically with TS 22.280, which provides the MCPTT requirements applicable by two or more mission critical services, referred to as "MCX" (Mission Critical X, with X = PTT or X = MCVideo or X = Data).

Overview of TS 22.280 on MCPTT Common Requirements

TS 22.280 follows the same overall structure as TS 22.179, the main clauses being: "on-network", "off-network" or common. TS 22.280 also specifies other types of MCX Service capabilities: Inter-MCX Service interworking, Air ground air communication and MCX Service in IOPS mode.

Some MCX service capabilities common to "on-network" and "off-network" are:

General Group Communication, Broadcast Group Communication, Late Communication Entry, Receiving from multiple MCX Service communication, Private Communication, MCX Service Priority, MCX Service User ID, MCX UE Management and User Profile, Support for multiple devices, Location, Security and Media quality.

Relay and Gateway capability, Control and management by Mission Critical Organizations, General administrative –groups and users, open interfaces for MCX Services, Media forwarding, Receipt notification and additional services for MCX Service communications.

Some MCX service capabilities specific to "on-network" are:

General administrative groups and users, MCX Service communications with notification and acknowledgement for Group Communications and Queuing, General capabilities, General MCX Service Group Communications and Broadcast Group Communication.

Dynamic Group management (i.e., dynamic regrouping), Private Communication, MCX Service Priority, IDs and aliases, MCX Service User Profile management, Support for multiple devices, Location, Security, Interactions for MCX Service Group and Private Communications, Additional Services (including Discreet and Ambient Listening, Remotely initiated Communication as well as recording and audit requirements).

Interaction with telephony services, Interworking ( including non-3GPP access and Interworking between MCX Service systems), MCX Service coverage extension using ProSe UE-to-Network Relays and Additional MCX Services related to Communication rejection and queuing.

Some MCX service capabilities specific to "off-network" are:

General off-network MCX Services, Admission control, Communication termination, Broadcast Group, MCX Service priority requirements, Communication types based on priority, Location, Security, Off-Network operations and UE functionality, Streaming for ProSe UE-to-UE and UE-to-Network Relay, Switching to off-network MCX Service, Off-network recording and audit, and Off-Network UE-to-UE Relay for Private and Group communications

Finally, here are some examples of Inter-MCX Service interworking requirements:

Concurrent operation of different MCX Services, Use of un-sharable resources within a UE, Single group with multiple MCX Services, Priority between services,

6.4 Mission Critical Video over LTE

Summary based on the input provided by Huawei in SP-170767.

Introduction

Clause 6.1 above explains the relationship in Rel-14 between TS 22.179 (MCPTT), TS 22.280 (MCCoRe), TS 22.281 (MCVideo) and TS 22.282 (MCData).

This clause deals more specifically with TS 22.281, which provides the MCPTT requirements applicable to MC Video, and the corresponding Stage 2 in 23.281.

The MCImp-MCVideo work item specifies the MCVideo service communications to support on-network and off-network operations. It defines the identities, procedures, information flows and related configurations corresponding to UE, user profile, group and service.

It specifies video media communication between several users (i.e. group call or private call), where each user has the ability to gain access to the permission to stream video in an arbitrated manner for on-network and off-network operations.

The Stage 2 (Architecture) for MCVideo is organized as the Stage 1, i.e.:

TS 23.280 specifies the common functional architecture aspects service including the common services core and the generic mechanisms like UE-to-network relay, affiliation and emergency alerts. These common aspects applies to MCVideo; and

TS 23.281, specifically dedicated to MCVideo. It specifies the MCVideo service functional architecture, procedures, information flows and related configuration information .

Overview of TS 22.281 on MCPTT Video

The MCVideo service capabilities specified for on-network operations are:

MCVideo group affiliation, MCVideo group de-affiliation and remote change of affiliation to MCVideo groups.

Group call setup, group call release, late entry to group call and re-joining a group call enabled for pre-arranged groups. Group call setup up, group call release, joining a group call and re-joining a group call enabled for chat groups. Exiting a group call due to de-affiliation is applicable for pre-arranged and chat group calls.

Emergency group call commencement, emergency group call cancel and upgrade of a group call to emergency group call.

Imminent peril group call commencement, imminent peril group call cancel and upgrade of a group call to imminent peril group call.

Emergency alert initiation and emergency state cancel (applicable for group calls)

Private call setup in automatic and manual commencement mode.

Transmission initiation and control, remotely initiated transmission, transmission revoke, transmission cancel, transmission queued, user or server initiated transmission cancel from queue and user or remote authorized user or server initiated transmission end.
Reception initiation and control, user or server initiated reception and mandatory or negotiated reception override.

Configurations for MCVideo service pertaining to UE, user profile, service and group.

User authentication (see TS 24.482) and MCVideo service authorization.

The MCVideo service capabilities specified for off-network operations are:

Group communication setup, active join to group communication, passive join to group communication and group communication release due to inactivity.

Emergency group communication commencement, emergency group communication cancel and upgrade of a group communication to emergency group communication.

Imminent peril group communication commencement, imminent peril group communication cancel and upgrade of a group communication to imminent peril group communication.

Emergency alert initiation and emergency state cancel (applicable for group calls).

Private call setup in automatic and manual commencement mode.

Granting a transmission, rejecting a transmission, releasing a transmission, overriding a transmission due to limit of maximum simultaneous transmissions reached and handling simultaneous transmissions.

Transmission control in single arbitration or self-arbitration modes, overriding a transmission of a single arbitrator and assignment of a new transmission arbitrator, releasing a transmission arbitrator and delegating a new transmission arbitrator when releasing.

Configurations for MCVideo service pertaining to UE, user profile, service and group. (see TS 24.481, TS 24.483 and TS 24.484)

The MCVideo service communications uses H.264 (AVC) codec as specified in TS 26.281. Based on operator or MCVideo service provider policy, the MCVideo service may optionally and additionally support the H.265 (HEVC) codec as specified in TS 26.281. If MCVideo service supports combined or separate handling of video and audio streams, then MCPTT audio codecs may be supported as specified in TS 26.179. The media transport protocols supported for MCVideo service are RTP and SRTP as specified in TS 26.281.
"""

In [13]:
text = QUERY.format(NUM_QUESTIONS=NUM_QUESTIONS, CONTEXT=CONTEXT)
chat = [
	{"role": "user", "content": text},
]
prompt = tokenizer.apply_chat_template(
	chat, tokenize=False, add_generation_prompt=True
)

In [14]:
inputs = tokenizer(
	prompt, return_tensors = "pt", padding=True, truncation=True
).to("cuda")

In [15]:
predicted = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2096,)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here are 5 questions based on the provided context:

<qa>
question: What is the purpose of TS 22.280 in the context of MCPTT?
option 1: To specify the requirements for MCPTT in Rel-14
option 2: To provide the architecture for MCPTT in Rel-14
option 3: To specify the requirements for MCVideo in Rel-14
option 4: To provide the architecture for MCVideo in Rel-14
option 5: To specify the requirements for MCData in Rel-14
answer: option 1: To specify the requirements for MCPTT in Rel-14
context: TS 22.280 provides the MCPTT requirements applicable by two or more mission critical services, referred to as "MCX" (Mission Critical X, with X = PTT or X = MCVideo or X = Data).
</qa>

<qa>
question: What is the main clause structure of TS 22.280?
option 1: "on-network", "off-network", and "common"
option 2: "on-network", "off-network", and "specific"
option 3: "on-network", "off-network", and "general"
option 4: "on-network", "off-network", and "common" and "specific"
option 5: "on-network", "off-

In [16]:
len(prompt)

9997